# 多帧图像转为视频

In [ ]:
import cv2
import os

# 图像文件夹路径
image_folder = "visualResult/CoDeFv1/beauty_1"
fps = 30.0

# 视频输出路径和名称
video_name = "visualResult/CoDeFv1/beauty_1.avi"

# 获取图像文件夹中的所有图像文件名
images = [img for img in os.listdir(image_folder)]

# 读取第一张图像，获取图像尺寸
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# 使用cv2.VideoWriter创建视频写入对象
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"MJPG"), fps, (width, height))

# 将每个图像逐帧写入视频
for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

# 关闭视频写入对象
video.release()
